In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from ay2.torch.deepfake_detection import DeepfakeAudioClassification

In [16]:
from wav2clip import get_model

In [2]:
from .wav2clip import get_model

## 测试

In [ ]:
model = get_model()
x = torch.randn(2, 48000)
model(x)

## Lit model

In [17]:
class Wav2Clip(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.model = get_model()
        self.proj = nn.Linear(512, 1)
        
    
    def forward(self, x):
        feat = self.model(x)
        return self.proj(feat), feat

In [12]:
class Wav2Clip_lit(DeepfakeAudioClassification):
    def __init__(self, backend="linear", **kwargs):
        super().__init__()
        self.model = Wav2Clip()
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.save_hyperparameters()
        
    def calcuate_loss(self, batch_res, batch):
        label = batch["label"]
        loss = self.loss_fn(batch_res["logit"], label.type(torch.float32))
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(), lr=0.0001, weight_decay=0.0001
        )
        return [optimizer]

    def _shared_pred(self, batch, batch_idx):
        audio, sample_rate = batch["audio"], batch["sample_rate"]
        if len(audio.shape) == 3:
            audio = audio[:, 0, :]

        out, feat = self.model(audio)
        out = out.squeeze(-1)
        batch_pred = (torch.sigmoid(out) + 0.5).int()
        return {
            "logit": out,
            "pred": batch_pred,
            "feature": feat
        }